In [145]:
import argparse
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.serialization import load_pem_public_key, load_pem_private_key
import os
import random

### 1. Генерация ключей гибридной системы

In [146]:
way = os.path.abspath('')
file_settings= {
    'initial_file': way + '\\' + 'initial_file.txt',
    'encrypted_file': way + '\\' + 'encrypted_file.txt',
    'decrypted_file': way + '\\' + 'decrypted_file.txt',
    'symmetric_key': way + '\\' + 'symmetric_key.txt',
    'public_key': way + '\\' + 'public_key.pem',
    'private_key': way + '\\' + 'private_key.pem',
}

keys = [i for i in range(40, 129, 8)]


генерация ключа для симметричного алгоритма

In [149]:
from cryptography.hazmat.primitives.asymmetric import padding
def gen_keys(file_settings: dict, keys: list):
    len_key = random.randint(0, len(keys) - 1)
    key = os.urandom(keys[len_key])
    print('Ключ симметричного шифрования:')
    print(type(key))
    print(key)
    
    assym_keys = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048
    )
    private_key = assym_keys
    public_key = assym_keys.public_key()
    
    print('\nЗакрытый ключ ассимметричного шифрования:')
    print(type(private_key))
    print(private_key)
    print('\nОтрытый ключ ассимметричного шифрования:')
    print(type(public_key))
    print(public_key)
    
    c_key = public_key.encrypt(key, padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()),algorithm=hashes.SHA256(),label=None))
    print('\nЗашифрованный ключ симметричного шифрования')
    print(c_key)
    
    with open(file_settings['public_key'], 'wb') as public_out:
            public_out.write(public_key.public_bytes(encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo))

    with open(file_settings['private_key'], 'wb') as private_out:
            private_out.write(private_key.private_bytes(encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.TraditionalOpenSSL,
            encryption_algorithm=serialization.NoEncryption()))
            
    c_key = public_key.encrypt(key, padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()),algorithm=hashes.SHA256(),label=None))
    with open(file_settings['symmetric_key'], 'wb') as key_file:
        key_file.write(c_key)

генерация пары ключей для acсимметричного алгоритма

In [150]:
gen_keys(file_settings, keys)

Ключ симметричного шифрования:
<class 'bytes'>
b'\xe9"k\x90D\x0c?\xd5\xc1)\xc9\xe3\xed\xf2.5ub\x95:\xd2U\xba\x8e\x1b\xdf\xce\xc406\xc0|\x19\xd7\xd05o\xa1\xc8-NY\x0f\xaa\x86\x99\xa8\xf1#\xe9\xe6`\xa1\xff\x89\x86\xcf\xbb\xe2\xe0\xca\xec\x17\xe1\x1e\xda\x10"c\xd6DO'

Закрытый ключ ассимметричного шифрования:
<class 'cryptography.hazmat.backends.openssl.rsa._RSAPrivateKey'>

Отрытый ключ ассимметричного шифрования:
<class 'cryptography.hazmat.backends.openssl.rsa._RSAPublicKey'>

Зашифрованный ключ симметричного шифрования
b'\xd1\xa9-Y\xb0bO))f\x8aX\xe8\xc4\x9e\xc0b\x0f\x0e\xb9\xb4<\xe1O2\xf1Tk%\xd2\n\xe7\x98P\xb0\x9f\xbf\x95fO\x80\xc0O\xde(C\xee\xc4\x15h\x96aN\xb6\x82\xa6\x9d\x12Um\xfc/\xefG\x8c\xbf\x12t=\x8dU\xf7\xff\xa93\xcc\xde\x1ba\xbd\x81\xd6E\xf6\x17\xaei\xb2\x98\xd8\x03\xd5e\x9c\x84\xf1\xde=\xf0\xd0\x9ftx\xa2\x95\x01+-J\x8fZ\xdfx\xc9\x99\xc8\rX\xcf\x05\xe1\x821\x18y\x82\xcbc#QJ\xb1\xc5\xca \xa7\xf3\\r\x12\x89\x98\x93\xc8e\x1b\xa4\xa0\xf9Ej\xd4\x06\xea\x0f\x02X/\xd9w9\xffd\xa2\xb31\

### 2. Шифрование данных гибридной системой

In [155]:
from cryptography.hazmat.primitives.asymmetric import padding
def key_dec(key, private_key):
    return private_key.decrypt(key,padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()),algorithm=hashes.SHA256(),label=None))

In [156]:
from cryptography.hazmat.primitives import padding
def f_padder():
    return padding.ANSIX923(32).padder()

In [157]:
def text_encryption(file_settings: dict):
    with open(file_settings['initial_file'], 'r', encoding = 'UTF-8') as file:
        text = file.read()
    print('Исходный текст: ')
    #print(text)
    
    with open(file_settings['private_key'], 'rb') as file:
        private_bytes = file.read()
    d_private_key = load_pem_private_key(private_bytes,password=None,)
    print('\nРасшифрованный симметричный ключ:')
    #print(type(d_private_key))
    #print(d_private_key)
    
    with open(file_settings['symmetric_key'], 'rb') as file:
        sym_key = file.read()
    
    print('Ключ симметричного шифрования:')
    #print(type(sym_key))
    #print(sym_key)
    
    dec_sym_key = key_dec(sym_key, d_private_key)
    print('\nРасшифрованный люч симметричного шифрования:')
    #print(dec_sym_key)
    
    padder = f_padder()
    padded_text = padder.update(text)+padder.finalize()

    print(text)
    print(padded_text)

In [158]:
text_encryption(file_settings)

Исходный текст: 

Расшифрованный симметричный ключ:
Ключ симметричного шифрования:


AttributeError: module 'cryptography.hazmat.primitives.padding' has no attribute 'OAEP'